In [60]:
params = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 256,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_generator': 512,
        'num_layers_generator': 2,
        'dropout_generator': 0.01,
        'learning_rate_generator': 5e-2, 
        'hidden_size_discriminator':512,
        'num_layers_discriminator':2,
        'dropout_discriminator': 0.01,
        'learning_rate_discriminator': 5e-2, 
        'epochs': 50,
        'use_Paul_distance': True}

paramsGen = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_generator': 512,
        'num_layers_generator': 2,
        'dropout_generator': 0.01,
        'learning_rate_generator': 1e-4, 
        'epochs': 50}

paramsGenOnData = {
        'print_every': 5,
        'plot_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size': 512,
        'num_layers': 2,
        'dropout': 0.05,
        'learning_rate': 5e-2, 
        'epochs': 50,
        'use_Paul_distance': True}

paramsDisInit = {
        'input_size':25,
        'hidden_size_discriminator':512,
        'num_layers_discriminator':2,
        'dropout_discriminator': 0.01}

paramsDisOnGen = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_discriminator': 512,
        'num_layers_discriminator': 2,
        'dropout_discriminator': 0.01,
        'learning_rate_discriminator': 5e-2, 
        'epochs': 50
}

paramsDisOnData = {
        'print_every': 5,
        'optimizer': "SGD",
        'lossFunction': "MSE",
        'model_type': "gru",
        'alphabet': 'a0',
        'sequence_lenght_in': 16, 
        'sequence_lenght_out': 16, 
        'using_cuda': True,
        'batch_size': 128,
        'shuffle': True, 
        'num_workers': 6,
        'hidden_size_discriminator': 256,
        'num_layers_discriminator': 2,
        'dropout_discriminator': 0.01,
        'learning_rate_discriminator': 5e-2, 
        'epochs': 50
}



#from GruGanClass import MYGRU
#from RnnGanClass import MYRNN
#from LstmGanClass import MYLSTM
import torch
import os
from utilities import chordUtil
from utilities.chordUtil import *
from utilities import dataImport



dropout_generator = params['dropout_generator']
model_type = params['model_type']
num_layers_generator = params['num_layers_generator']
hidden_size_generator = params['hidden_size_generator']
alphabet = params['alphabet']
sequence_lenght_in = params['sequence_lenght_in']
sequence_lenght_out = params['sequence_lenght_out']
num_layers_discriminator = params['num_layers_discriminator']
hidden_size_discriminator = params['hidden_size_discriminator']

#dropoutstr = str(dropout).replace('.',',')
model_string_generator = "models/"+model_type+str(num_layers_generator)+"layers"+str(hidden_size_generator)+"blocks"+alphabet+"alphabet"+str(sequence_lenght_in)+"lenSeq_in"+str(sequence_lenght_out)+"lenSeq_out.pt"
model_string_discriminator = "models/"+model_type+str(num_layers_discriminator)+"layers"+str(hidden_size_discriminator)+"blocks"+alphabet+"alphabet"+str(sequence_lenght_out)+"lenSeq_out.pt"

# Getting alphabet size :
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
dictChord, listChord = chordUtil.getDictChord(eval(alphabet))
alphabet_size = len(dictChord)
#print(dictChord)

In [130]:
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
dictChord, listChord = chordUtil.getDictChord(eval(alphabet))
print(len(dictChord))
print(dictChord)

# Create datasets
#files_train ,files_test = train_test_split(filenames,test_size=0.7)
#if training:
#    dataset_train = dataImport.ChordSeqDataset(files_train, rootname, alphabet, dictChord, sequence_lenght)
#dataset_test = dataImport.ChordSeqDataset(files_test, rootname, alphabet, dictChord, sequence_lenght)

#dataset = dataImport.ChordSeqDataset(filenames, rootname, alphabet, dictChord, sequence_lenght_in+sequence_lenght_out)
#params["dataset"] = dataset

25
{'A:min': 0, 'C#:min': 1, 'A:maj': 2, 'G:min': 3, 'A#:min': 4, 'E:min': 5, 'B:maj': 6, 'G#:min': 7, 'F#:min': 8, 'F:min': 9, 'C:min': 10, 'B:min': 11, 'C:maj': 12, 'N': 13, 'C#:maj': 14, 'G#:maj': 15, 'F:maj': 16, 'D#:min': 17, 'E:maj': 18, 'D:min': 19, 'D:maj': 20, 'A#:maj': 21, 'F#:maj': 22, 'D#:maj': 23, 'G:maj': 24}


In [62]:
import torch
from random import randint
from utilities import chordUtil
from utilities import dataImport
from utilities import plotAndTimeUtil
from utilities.chordUtil import *
from utilities.dataImport import *
from utilities.plotAndTimeUtil import *
from sklearn.model_selection import train_test_split
import os
import numpy as np
import random
import sys
import torch.nn as nn
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from utilities import chordsDistances
from utilities.chordsDistances import getPaulMatrix
from utilities import remapChordsToBase
from utilities.remapChordsToBase import remapPaulToTristan
from random import choices


class MYGRU(nn.Module):

    # Defining Pytorch things to make the Network

    def __init__(self, input_size, hidden_size_generator, num_layers_generator, dropout_generator):
        super(MYGRU, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size_generator, num_layers = num_layers_generator, batch_first = True, dropout = dropout_generator)
        self.last_fully_connected = nn.Linear(hidden_size_generator, input_size)
        # not sure about the dim = 1 in softmax
        self.softmax = nn.LogSoftmax(dim=1)

        
    # Defining some cool functions to make work easier and trop stylé

    # About the trainingData :
    # 0 is training number
    # 1 is optimizer used
    # 2 is loss used
    # 3 is loss values on train set
    # 4 is loss values on test set
    # 5 is accuracy on train set
    # 6 is accuracy on test set
    # 7 is time
    # 8 is learning rate
    # 9 is use_Paul_distance

        
        # Usefull for monitoring
        self.trainingData = [[0],["None"],["None"],[[0]],[[0]],[[0]],[[0]],[0],[0],["No"]]


    def forward(self, input_batch):
        output, hidden = self.rnn(input_batch)
        output = output[:,-1,:]
        #print("outputsize : ")
        #print(output.size())
        output = self.last_fully_connected(output)
        #print("outputsize : ")
        #print(output.size())
        # not sure about the dim = 1 in softmax
        output = self.softmax(output)
        #print("outputsize : ")
        #print(output.size())
        return output
    
    
    def trainAndTest(self, model_type, print_every, plot_every, optimizer, lossFunction, alphabet='a0', sequence_lenght=16, using_cuda=True, batch_size=128, shuffle=True, num_workers=6, hidden_size=128, num_layers=2, dropout=0.1, learning_rate=1e-4, epochs=10, use_Paul_distance=False):
        if using_cuda:
            print("Trying using Cuda ...")
            use_cuda = torch.cuda.is_available()
            if use_cuda:
                print("OK")
            else:
                print("Woops, Cuda cannot be found :'( ")
            device = torch.device("cuda:0" if use_cuda else "cpu")
        else:
            device = torch.device("cpu")
            print("Using Cpu")
        self.to(device)

        start = time.time()

        all_losses, test_losses, accuracy_train, accuracy_test = self.doEpochs(
            model_type, epochs, print_every, plot_every, optimizer, lossFunction, learning_rate, batch_size, shuffle, num_workers, alphabet, sequence_lenght, device, start, use_Paul_distance, training=True)

        self.trainingData[0].append(self.trainingData[0][-1]+1)
        self.trainingData[1].append(optimizer)
        self.trainingData[2].append(lossFunction)
        self.trainingData[3].append(all_losses)
        self.trainingData[4].append(test_losses)
        self.trainingData[5].append(accuracy_train)
        self.trainingData[6].append(accuracy_test)
        self.trainingData[7].append(plotAndTimeUtil.timeSince(start))
        self.trainingData[8].append(learning_rate)
        self.trainingData[9].append(use_Paul_distance)

        print("Finished Training")

        return

    def doEpochs(self, model_type, epochs, print_every, plot_every, optimizer, lossFunction, learning_rate, batch_size, shuffle, num_workers, alphabet, sequence_lenght, device, start, use_Paul_distance, training=True):

        # Init Training results and monitoring data
        all_losses = []
        test_losses = []
        total_loss = 0  # Reset every plot_every iters
        test_loss = 0
        correct_guess_train, wrong_guess_train, correct_guess_test, wrong_guess_test = 0, 0, 0, 0
        accuracy_test = []
        accuracy_train = []

        rootname = "inputs/jazz_xlab/"
        filenames = os.listdir(rootname)
        dictChord, listChord = chordUtil.getDictChord(eval(alphabet))

        # Create datasets
        files_train, files_test = train_test_split(filenames, test_size=0.7)
        if training:
            dataset_train = dataImport.ChordSeqDataset(
                files_train, rootname, alphabet, dictChord, sequence_lenght)
        dataset_test = dataImport.ChordSeqDataset(
            files_test, rootname, alphabet, dictChord, sequence_lenght)

        # Create generators
        params = {'batch_size': batch_size,
                  'shuffle': shuffle,
                  'num_workers': num_workers}

        # Get Paul distance matrix :
        if use_Paul_distance:
            M = chordsDistances.getPaulMatrix()
            M = remapChordsToBase.remapPaulToTristan(M, alphabet)
            distance_tensor = torch.tensor(M, dtype=torch.float).to(device)
        else:
            distance_tensor = 0

        if training:
            training_generator = data.DataLoader(dataset_train, **params)
        testing_generator = data.DataLoader(dataset_test, **params)

        # TODO Put more optimisers
        if optimizer == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        if optimizer == "SGD":
            optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
        else:
            raise ValueError("This optimizer is unknown to me")

        # TODO Put more losses
        if lossFunction == "CrossEntropy":
            criterion = nn.CrossEntropyLoss()
        if lossFunction == "MSE":
            criterion = nn.MSELoss()
        else:
            raise ValueError("This loss function is unknown to me")

        if training:
            print("Start training")
        for epoch in range(1, epochs):
            # Training
            if training:
                self.train(mode=True)
                for local_batch, local_labels in training_generator:
                    output, loss, correct_guess, wrong_guess = self.oneBatchTrain(
                        local_batch, local_labels, optimizer, criterion, device, use_Paul_distance, distance_tensor)
                    total_loss += loss
                    correct_guess_train += correct_guess
                    wrong_guess_train += wrong_guess

                if epoch % print_every == 0:
                    accuracy = correct_guess_train / \
                        (correct_guess_train+wrong_guess_train)
                    print('Train : %s (%d %d%%) loss : %.4f, accuracy : %.4f%%' % (
                        plotAndTimeUtil.timeSince(start), epoch, epoch / epochs * 100, loss, accuracy*100))

                if epoch % plot_every == 0:
                    accuracy = correct_guess_train / \
                        (correct_guess_train+wrong_guess_train)
                    all_losses.append(total_loss / (plot_every))
                    accuracy_train.append(accuracy*100)
                    total_loss = 0
                    correct_guess_train, wrong_guess_train = 0, 0

            # Testing
            self.train(mode=False)
            for local_batch, local_labels in testing_generator:
                output, loss, correct_guess, wrong_guess = self.oneBatchTest(
                    local_batch, local_labels, optimizer, criterion, device, use_Paul_distance, distance_tensor)
                test_loss += loss

                correct_guess_test += correct_guess
                wrong_guess_test += wrong_guess

            if epoch % print_every == 0:
                accuracy = correct_guess_test / \
                    (correct_guess_test+wrong_guess_test)
                if training:
                    print('Test : %s (%d %d%%) loss : %.4f, accuracy : %.4f%%' % (
                        plotAndTimeUtil.timeSince(start), epoch, epoch / epochs * 100, loss, accuracy*100))

            if epoch % plot_every == 0:
                accuracy = correct_guess_test / \
                    (correct_guess_test+wrong_guess_test)
                test_losses.append(test_loss / (plot_every))
                accuracy_test.append(accuracy*100)
                test_loss = 0
                correct_guess_test, wrong_guess_test = 0, 0

        if training:
            return all_losses, test_losses, accuracy_train, accuracy_test
        else:
            return test_losses, accuracy_test


# define Train on one batch function

    def oneBatchTrain(self, local_batch, local_labels, optimizer, criterion, device, use_Paul_distance, distance_tensor):
        optimizer.zero_grad()
        loss = 0
        correct_guess, wrong_guess = 0, 0

        # if tensor of shape 1 in loss function (ex : CrossEntropy)
        #local_labels_argmax = torch.tensor(
        #    [torch.argmax(local_label) for local_label in local_labels]).to(device)

        local_batch, local_labels = local_batch.to(
            device), local_labels.to(device)
        output = self.forward(local_batch)

        for i in range(len(local_batch)):
            if torch.argmax(output[i]) == torch.argmax(local_labels[i]):
                correct_guess += 1
            else:
                wrong_guess += 1

        # using Paul Distance
        if use_Paul_distance:
            loss_mult_coeff = 0
            for i in range(len(local_batch)):
                loss_mult_coeff += torch.matmul(distance_tensor, local_labels[i])[
                    torch.argmax(output[i])]

            loss_mult_coeff = loss_mult_coeff/len(local_batch)
        else:
            loss_mult_coeff = 1

        loss = loss_mult_coeff * criterion(output, local_labels)
        loss.backward()
        optimizer.step()

        return output, loss.item() / len(local_batch), correct_guess, wrong_guess


# Defining test on one batch function

    def oneBatchTest(self, local_batch, local_labels, optimizer, criterion, device, use_Paul_distance, distance_tensor):
        loss = 0
        correct_guess, wrong_guess = 0, 0

        # if tensor of shape 1 in loss function (ex : CrossEntropy)
        #local_labels_argmax = torch.tensor(
        #    [torch.argmax(local_label) for local_label in local_labels]).to(device)
        local_batch, local_labels = local_batch.to(
            device), local_labels.to(device)
        output = self.forward(local_batch)

        for i in range(len(local_batch)):
            if torch.argmax(output[i]) == torch.argmax(local_labels[i]):
                correct_guess += 1
            else:
                wrong_guess += 1

        # using Paul Distance
        if use_Paul_distance:
            loss_mult_coeff = 0
            for i in range(len(local_batch)):
                loss_mult_coeff += torch.matmul(distance_tensor, local_labels[i])[
                    torch.argmax(output[i])]

            loss_mult_coeff = loss_mult_coeff/len(local_batch)
        else:
            loss_mult_coeff = 1

        loss = criterion(output, local_labels)

        return output, loss.item() / len(local_batch), correct_guess, wrong_guess


    def trainOnDis(self, model_type, print_every, optimizer, lossFunction, disNet, alphabet='a0', sequence_lenght_in=16, sequence_lenght_out=16, using_cuda=True, batch_size=128, shuffle=True, num_workers=6, hidden_size_generator=128, num_layers_generator=2, dropout_generator=0.1, learning_rate_generator=1e-4, epochs=10, sampling=True):
        #disNet.train(mode=False)
        self.train(mode=True)
        
        if using_cuda:
            print("Trying using Cuda ...")
            use_cuda = torch.cuda.is_available()
            if use_cuda:
                print("OK")
            else:
                print("Woops, Cuda cannot be found :'( ")
            device = torch.device("cuda:0" if use_cuda else "cpu")
        else:
            device = torch.device("cpu")
            print("Using Cpu")
        self.to(device)

        start = time.time()

        all_losses, similarity = self.doEpochsOnDis(model_type, epochs, print_every, optimizer, lossFunction, learning_rate_generator, batch_size, shuffle, num_workers, alphabet, sequence_lenght_in, sequence_lenght_out, device, start, disNet, sampling, training=True)
   
        print("Finished Training")
        return similarity

    def doEpochsOnDis(self, model_type, epochs, print_every, optimizer, lossFunction, learning_rate_generator, batch_size, shuffle, num_workers, alphabet, sequence_lenght_in, sequence_lenght_out, device, start, disNet, sampling, training=True):

        # Init Training results and monitoring data
        all_losses = []
        total_loss = 0 # Reset every plot_every iters
        genWin = 0
        totalSize = 0
        genWinTest = []

        # Creating Dataset
        dataset = dataImport.ChordSeqDataset(filenames, rootname, alphabet, dictChord, sequence_lenght_in+sequence_lenght_out)
        
        # Create generators
        params = {'batch_size': batch_size,
                  'shuffle': shuffle,
                  'num_workers': num_workers}
            
        if training:
            training_generator = data.DataLoader(dataset, **params)

        #TODO Put more optimisers
        if optimizer == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = learning_rate_generator)
        elif optimizer =="SGD":
            optimizer = torch.optim.SGD(self.parameters(), lr = learning_rate_generator)
        else:
            raise ValueError("This optimizer is unknown to me")

        #TODO Put more losses
        if lossFunction == "CrossEntropy":
            criterion = nn.CrossEntropyLoss()
        elif lossFunction == "MSE":
            criterion = nn.MSELoss()
        else:
            raise ValueError("This loss function is unknown to me")

        if training:
            print("Start training")
            
        for epoch in range(epochs):
            # Training
            if training:
                self.train(mode=True)
                for local_batch, local_labels in training_generator:
                    output, loss, genWinOne = self.oneBatchTrainOnDis(local_batch, local_labels, optimizer, criterion, device, sequence_lenght_in, sequence_lenght_out, sampling, disNet)
                    total_loss += loss
                    genWin += genWinOne
                    totalSize += len(local_batch)

                if epoch % print_every == 0 and epoch != 0:
                    genWin = genWin/totalSize
                    total_loss = total_loss/totalSize
                    genWinTest.append(genWin)
                    all_losses.append(total_loss)
                    print('genWin : %s (%d %d%%) loss : %.4f, accuracy : %.4f%%' % (plotAndTimeUtil.timeSince(start), epoch, epoch / epochs * 100, 100*total_loss, genWin*100))
                    genWin = 0
                    total_loss = 0
                    totalSize = 0
                    
            # Testing
            #self.train(mode=False)

        return all_losses, genWinTest



# define Train on one batch function

    def oneBatchTrainOnDis(self, local_batch, local_labels, optimizer, criterion, device, sequence_lenght_in, sequence_lenght_out, sampling, disNet):
       
        optimizer.zero_grad()
        loss = 0
        correct_guess = 0
        softmax = nn.Softmax(dim=1)
        temperature = 2


        # if tensor of shape 1 in loss function (ex : CrossEntropy)
        #local_labels_argmax = torch.tensor([torch.argmax(local_label) for local_label in local_labels]).to(device)
        

        #local_labels = local_labels.to(device)
        working_batch = torch.zeros([len(local_batch),sequence_lenght_in+sequence_lenght_out,len(local_batch[0,0])])
        working_batch[:,0:sequence_lenght_in,:] = local_batch[:,0:sequence_lenght_in,:]
        local_batch = local_batch.to(device)

        for i in range(sequence_lenght_out):
            output = self.forward(local_batch)
            #local_batch[:,0:sequence_lenght_in-1,:] = local_batch[:,1:sequence_lenght_in,:]
            
            if sampling:
                #choice = choices(range(len(listChord)),softmax(output[0]))[0]
                #print(output.size())
                output = softmax(output)
                _output = output.cpu().div(temperature).exp().data

                #print(output.size())
                topi = torch.multinomial(_output, 1)
                #print(topi.size())
                #print(working_batch.size())
                for k in range(len(local_batch)):
                    working_batch[k,sequence_lenght_in+i, topi[k,0].item()] = 1
                #print(working_batch[0])
            else:
                #TODO
                generated_sequence[sequence_lenght_in] = listChord[torch.argmax(output_probability).item()]
                local_batch[:, sequence_lenght_in, torch.argmax(output[:,:]).item()] = 1                
                
        #print(working_batch.size())
        #local_batch has been transformed in output
        disNet.to(device)
        disDecision = disNet(working_batch[:,sequence_lenght_in:,:].to(device))
        
        #TODO : change that 0 in something more relevant
        
        fooling = 0
        for i in range(len(local_batch)):
            if disDecision[i,0].item() > 0.5:
                fooling+=1
        #print(disDecision.size())
        loss = criterion(disDecision, torch.ones([len(local_batch),1], dtype=torch.float).to(device))
        loss.backward()
        optimizer.step()

        return output, loss.item(), fooling

    def plotLastTraining(self):
        plotAndTimeUtil.PlotResults(self.trainingData[3][-1], self.trainingData[4][-1], self.trainingData[5][-1], self.trainingData[6][-1])

    def plotAllTraining(self):
        plotAndTimeUtil.PlotAllResults(self.trainingData)

    def toString(self, model_type, print_every, plot_every, optimizer, lossFunction, alphabet='a0', sequence_lenght=16, using_cuda=True, batch_size=128, shuffle=True, num_workers=6, hidden_size=128, num_layers=2, dropout=0.1, learning_rate=1e-4, epochs=10, use_Paul_distance=False):
        #dropoutstr = str(dropout).replace('.',',')
        model_string = "models/"+model_type+str(num_layers)+"layers"+str(hidden_size)+"blocks"+alphabet+"alphabet"+str(sequence_lenght)+"lenSeq.pt"
        return model_string
    
    def generateFromSequence(self, test_sequence, generation_lenght, alphabet, sampling=False, using_cuda=True, silent=True):
        lenSeq = len(test_sequence)
        
        # Cuda blabla
        if using_cuda:
            print("Trying using Cuda ...")
            use_cuda = torch.cuda.is_available()
            if use_cuda:
                print("OK")
            else:
                print("Woops, Cuda cannot be found :'( ")
            device = torch.device("cuda:0" if use_cuda else "cpu")
        else:
            device = torch.device("cpu")
            print("Using Cpu")
        self.to(device)    
        
        # Getting chords dictionary
        rootname = "inputs/jazz_xlab/"
        filenames = os.listdir(rootname)
        dictChord, listChord = chordUtil.getDictChord(eval(alphabet))
        
        
        # Initialising objects
        test_sequence_tensor = torch.zeros(1, len(test_sequence), len(dictChord)).to(device)
        last_chords_output = torch.zeros(1, lenSeq, len(dictChord)).to(device)
        test_sequence_tensor.requires_grad = False
        last_chords_output.requires_grad = False
        for t in range(len(test_sequence)):
            test_sequence_tensor[0, t, dictChord[test_sequence[t]]] = 1
            if t != len(test_sequence)-1 :
                last_chords_output[0, t-1, dictChord[test_sequence[t]]] = 1

                
        generated_sequence = [0 for i in range(generation_lenght)]
        generated_sequence[0:lenSeq] = test_sequence

        self.train(mode=False)
        softmax = nn.Softmax(dim=0)

        for t in range(generation_lenght-lenSeq):
            if t == 0:
                output_probability = self(test_sequence_tensor)

                if sampling:
                    choice = choices(range(len(listChord)),softmax(output_probability[0]))[0]
                    generated_sequence[t+lenSeq] = listChord[choice]
                    last_chords_output[0, lenSeq-1, choice] = 1

                else: 
                    generated_sequence[t+lenSeq] = listChord[torch.argmax(output_probability).item()]
                    last_chords_output[0, lenSeq-1, torch.argmax(output_probability).item()] = 1


            else:

                last_chords_output.to(device)        
                output_probability = self(last_chords_output)
                last_chords_output[0,0:lenSeq-1] = last_chords_output[0,1:lenSeq]

                if sampling:
                    choice = choices(range(len(listChord)),softmax(output_probability[0]))[0]
                    generated_sequence[t+lenSeq] = listChord[choice]
                    last_chords_output[0, lenSeq-1, choice] = 1

                else:
                    last_chords_output[0, lenSeq-1, torch.argmax(output_probability).item()] = 1
                    generated_sequence[t+lenSeq] = listChord[torch.argmax(output).item()]


        for i in range(generation_lenght):
            if i%4 == 0:
                print(generated_sequence[i:i+4])
            if i == lenSeq-1 :
                print("generated :")
                
        if silent:
            return
        else:
            return generated_sequence
        

In [63]:
if model_type == "gru":
    myGenerator = MYGRU(alphabet_size, hidden_size_generator, num_layers_generator, dropout_generator)
elif model_type == "lstm":
    myGenerator = MYLSTM(alphabet_size, hidden_size_generator, num_layers_generator, dropout_generator)
elif model_type == "rnn":
    myGenerator = MYRNN(alphabet_size, hidden_size_generator, num_layers_generator, dropout_generator)

In [64]:
#myNetwork = torch.load(model_string)

In [65]:
class MYDis(nn.Module):

    # Defining Pytorch things to make the Network

    def __init__(self, input_size, hidden_size_discriminator, num_layers_discriminator, dropout_discriminator):
        super(MYDis, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size_discriminator, num_layers = num_layers_discriminator, batch_first = True, dropout = dropout_discriminator)
        self.last_fully_connected = nn.Linear(hidden_size_discriminator, 1)
        # not sure about the dim = 1 in softmax
        #self.softmax = nn.LogSoftmax(dim=1)


        
        # Usefull for monitoring
        #self.trainingData = [[0],["None"],["None"],[[0]],[[0]],[[0]],[[0]],[0],[0],["No"]]


    def forward(self, input_batch):
        output, hidden = self.rnn(input_batch)
        output = output[:,-1,:]
        #print("outputsize : ")
        #print(output.size())
        output = self.last_fully_connected(output)
        #print("outputsize : ")
        #print(output.size())
        # not sure about the dim = 1 in softmax
        #output = self.softmax(output)
        #print("outputsize : ")
        #print(output.size())
        return output
    

    def trainOnGen(self, model_type, print_every, optimizer, lossFunction, genNet, alphabet='a0', sequence_lenght_in=16, sequence_lenght_out=16, using_cuda=True, batch_size=128, shuffle=True, num_workers=6, hidden_size_discriminator=128, num_layers_discriminator=2, dropout_discriminator=0.1, learning_rate_discriminator=1e-4, epochs=10, sampling=True):
        self.train(mode=True)
        genNet.train(mode=False)
        if using_cuda:
            print("Trying using Cuda ...")
            use_cuda = torch.cuda.is_available()
            if use_cuda:
                print("OK")
            else:
                print("Woops, Cuda cannot be found :'( ")
            device = torch.device("cuda:0" if use_cuda else "cpu")
        else:
            device = torch.device("cpu")
            print("Using Cpu")
        self.to(device)

        start = time.time()

        all_losses, similarity = self.doEpochsOnGen(model_type, epochs, print_every, optimizer, lossFunction, learning_rate_discriminator, batch_size, shuffle, num_workers, alphabet, sequence_lenght_in, sequence_lenght_out, device, start, genNet, sampling, training=True)
   
        print("Finished Training")
        return similarity

    def doEpochsOnGen(self, model_type, epochs, print_every, optimizer, lossFunction, learning_rate_discriminator, batch_size, shuffle, num_workers, alphabet, sequence_lenght_in, sequence_lenght_out, device, start, genNet, sampling, training=True):

        # Init Training results and monitoring data
        all_losses = []
        total_loss = 0 # Reset every plot_every iters
        disOnGenWin = 0
        totalSize = 0
        disOnGenWinTest = []

        # Creating Dataset
        dataset = dataImport.ChordSeqDataset(filenames, rootname, alphabet, dictChord, sequence_lenght_in+sequence_lenght_out)
        
        # Create generators
        params = {'batch_size': batch_size,
                  'shuffle': shuffle,
                  'num_workers': num_workers}
            
        if training:
            training_generator = data.DataLoader(dataset, **params)

        #TODO Put more optimisers
        if optimizer == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = learning_rate_discriminator)
        elif optimizer =="SGD":
            optimizer = torch.optim.SGD(self.parameters(), lr = learning_rate_discriminator)
        else:
            raise ValueError("This optimizer is unknown to me")

        #TODO Put more losses
        if lossFunction == "CrossEntropy":
            criterion = nn.CrossEntropyLoss()
        elif lossFunction == "MSE":
            criterion = nn.MSELoss()
        else:
            raise ValueError("This loss function is unknown to me")

        if training:
            print("Start training")
            
        for epoch in range(epochs):
            # Training
            if training:
                self.train(mode=True)
                for local_batch, local_labels in training_generator:
                    output, loss, disOnGenWinOne = self.oneBatchTrainOnGen(local_batch, local_labels, optimizer, criterion, device, sequence_lenght_in, sequence_lenght_out, sampling, genNet)
                    total_loss += loss
                    disOnGenWin += disOnGenWinOne
                    totalSize += len(local_batch)

                if epoch % print_every == 0 and epoch != 0:
                    disOnGenWin = disOnGenWin/totalSize
                    total_loss = total_loss/totalSize
                    disOnGenWinTest.append(disOnGenWin)
                    all_losses.append(total_loss)
                    print('disOnGen : %s (%d %d%%) loss : %.4f, accuracy : %.4f%%' % (plotAndTimeUtil.timeSince(start), epoch, epoch / epochs * 100, total_loss*100, disOnGenWin*100))
                    disOnGenWin = 0
                    total_loss = 0
                    totalSize = 0

                    
            # Testing
            #self.train(mode=False)

        return all_losses, disOnGenWinTest



# define Train on one batch function

    def oneBatchTrainOnGen(self, local_batch, local_labels, optimizer, criterion, device, sequence_lenght_in, sequence_lenght_out, sampling, genNet):
       
        optimizer.zero_grad()
        loss = 0
        correct_guess = 0
        softmax = nn.Softmax(dim=1)
        temperature = 2


        # if tensor of shape 1 in loss function (ex : CrossEntropy)
        #local_labels_argmax = torch.tensor([torch.argmax(local_label) for local_label in local_labels]).to(device)
        

        #local_labels = local_labels.to(device)
        working_batch = torch.zeros([len(local_batch),sequence_lenght_in+sequence_lenght_out,len(local_batch[0,0])])
        working_batch[:,0:sequence_lenght_in,:] = local_batch[:,0:sequence_lenght_in,:]
        local_batch = local_batch.to(device)

        for i in range(sequence_lenght_out):
            output = genNet.forward(local_batch)
            #local_batch[:,0:sequence_lenght_in-1,:] = local_batch[:,1:sequence_lenght_in,:]
            
            if sampling:
                #choice = choices(range(len(listChord)),softmax(output[0]))[0]
                #print(output.size())
                output = softmax(output)
                _output = output.cpu().div(temperature).exp().data

                #print(output.size())
                topi = torch.multinomial(_output, 1)
                #print(topi.size())
                #print(working_batch.size())
                for k in range(len(local_batch)):
                    working_batch[k,sequence_lenght_in+i, topi[k,0].item()] = 1
                #print(working_batch[0])
            else:
                #TODO
                generated_sequence[sequence_lenght_in] = listChord[torch.argmax(output_probability).item()]
                local_batch[:, sequence_lenght_in, torch.argmax(output[:,:]).item()] = 1                
                
        #print(working_batch.size())
        #local_batch has been transformed in output
        self.to(device)
        disDecision = self(working_batch[:,sequence_lenght_in:,:].to(device))
        
        #TODO : change that 0 in something more relevant
        fooling = 0
        for i in range(len(local_batch)):
            if disDecision[i,0].item() < 0.5:
                fooling+=1
        #print(disDecision.size())
        loss = criterion(disDecision, torch.zeros([len(local_batch),1], dtype=torch.float).to(device))
        loss.backward()
        optimizer.step()

        return output, loss.item(), fooling

    def trainOnData(self, model_type, print_every, optimizer, lossFunction, alphabet='a0', sequence_lenght_in=16, sequence_lenght_out=16, using_cuda=True, batch_size=128, shuffle=True, num_workers=6, hidden_size_discriminator=128, num_layers_discriminator=2, dropout_discriminator=0.1, learning_rate_discriminator=1e-4, epochs=10, sampling=True):
        self.train(mode=True)
        if using_cuda:
            print("Trying using Cuda ...")
            use_cuda = torch.cuda.is_available()
            if use_cuda:
                print("OK")
            else:
                print("Woops, Cuda cannot be found :'( ")
            device = torch.device("cuda:0" if use_cuda else "cpu")
        else:
            device = torch.device("cpu")
            print("Using Cpu")
        self.to(device)

        start = time.time()

        all_losses, similarity = self.doEpochsOnData(model_type, epochs, print_every, optimizer, lossFunction, learning_rate_discriminator, batch_size, shuffle, num_workers, alphabet, sequence_lenght_in, sequence_lenght_out, device, start, sampling, training=True)
   
        print("Finished Training")
        return similarity

    def doEpochsOnData(self, model_type, epochs, print_every, optimizer, lossFunction, learning_rate_discriminator, batch_size, shuffle, num_workers, alphabet, sequence_lenght_in, sequence_lenght_out, device, start, sampling, training=True):

        # Init Training results and monitoring data
        all_losses = []
        total_loss = 0 # Reset every plot_every iters
        disOnDataWin = 0
        totalSize = 0
        disOnDataWinTest = []

        # Creating Dataset
        dataset = dataImport.ChordSeqDataset(filenames, rootname, alphabet, dictChord, sequence_lenght_out)
        
        # Create generators
        params = {'batch_size': batch_size,
                  'shuffle': shuffle,
                  'num_workers': num_workers}
            
        if training:
            training_generator = data.DataLoader(dataset, **params)

        #TODO Put more optimisers
        if optimizer == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = learning_rate_discriminator)
        elif optimizer =="SGD":
            optimizer = torch.optim.SGD(self.parameters(), lr = learning_rate_discriminator)
        else:
            raise ValueError("This optimizer is unknown to me")

        #TODO Put more losses
        if lossFunction == "CrossEntropy":
            criterion = nn.CrossEntropyLoss()
        elif lossFunction == "MSE":
            criterion = nn.MSELoss()
        else:
            raise ValueError("This loss function is unknown to me")

        if training:
            print("Start training")
            
        for epoch in range(epochs):
            # Training
            if training:
                self.train(mode=True)
                for local_batch, local_labels in training_generator:
                    loss, disOnDataWinOne = self.oneBatchTrainOnData(local_batch, local_labels, optimizer, criterion, device, sequence_lenght_in, sequence_lenght_out, sampling)
                    total_loss += loss
                    disOnDataWin += disOnDataWinOne
                    totalSize += len(local_batch)

                if epoch % print_every == 0 and epoch != 0:
                    disOnDataWin = disOnDataWin/totalSize
                    total_loss = total_loss/totalSize
                    disOnDataWinTest.append(disOnDataWin)
                    all_losses.append(total_loss)
                    print('disOnDataWin : %s (%d %d%%) loss : %.4f, accuracy : %.4f%%' % (plotAndTimeUtil.timeSince(start), epoch, epoch / epochs * 100, total_loss*100, disOnDataWin*100))
                    disOnDataWin = 0
                    total_loss = 0
                    totalSize = 0

                    
            # Testing
            #self.train(mode=False)

        return all_losses, disOnDataWinTest



# define Train on one batch function

    def oneBatchTrainOnData(self, local_batch, local_labels, optimizer, criterion, device, sequence_lenght_in, sequence_lenght_out, sampling):
       
        optimizer.zero_grad()
        loss = 0
        correct_guess = 0
        softmax = nn.Softmax(dim=1)
        temperature = 2


        self.to(device)
        disDecision = self(local_batch.to(device))
        
        #TODO : change that 0 in something more relevant
        fooling = 0
        for i in range(len(local_batch)):
            if disDecision[i,0].item() > 0.5:
                fooling+=1
        #print(disDecision.size())
        loss = criterion(disDecision, torch.ones([len(local_batch),1], dtype=torch.float).to(device))
        loss.backward()
        optimizer.step()

        return loss.item(), fooling


    

    

In [66]:
myDis = MYDis(**paramsDisInit)

In [126]:
paramsGenOnData['epochs']=500
paramsGenOnData['print_every']=20
myGenerator.trainAndTest(**paramsGenOnData)

Trying using Cuda ...
OK
Start training
Train : 0m 26s (20 4%) loss : 0.3024, accuracy : 44.6893%
Test : 0m 27s (20 4%) loss : 0.1460, accuracy : 45.7702%
Train : 0m 54s (40 8%) loss : 0.2699, accuracy : 45.8851%
Test : 0m 55s (40 8%) loss : 0.1460, accuracy : 47.3457%
Train : 1m 22s (60 12%) loss : 0.2773, accuracy : 46.5651%
Test : 1m 23s (60 12%) loss : 0.1460, accuracy : 46.7737%
Train : 1m 50s (80 16%) loss : 0.2419, accuracy : 46.8230%
Test : 1m 51s (80 16%) loss : 0.1460, accuracy : 46.6433%
Train : 2m 18s (100 20%) loss : 0.2788, accuracy : 45.0176%
Test : 2m 18s (100 20%) loss : 0.1460, accuracy : 46.8339%
Train : 2m 46s (120 24%) loss : 0.2463, accuracy : 46.0727%
Test : 2m 47s (120 24%) loss : 0.1460, accuracy : 47.5163%
Train : 3m 19s (140 28%) loss : 0.2728, accuracy : 46.5885%
Test : 3m 20s (140 28%) loss : 0.1460, accuracy : 47.3256%
Train : 3m 48s (160 32%) loss : 0.3024, accuracy : 46.8933%
Test : 3m 49s (160 32%) loss : 0.1460, accuracy : 46.8941%
Train : 4m 17s (180 

In [129]:
paramsGen['epochs']=50
paramsGen['print_every']=5
paramsGen['learning_rate_generator']=1e-0
paramsGen['dropout_generator']=0.1
paramsGen['disNet']=myDis

myGenerator.trainOnDis(**paramsGen)

Trying using Cuda ...
OK
Start training
genWin : 0m 43s (5 10%) loss : 0.2031, accuracy : 39.8513%
genWin : 1m 18s (10 20%) loss : 0.2031, accuracy : 39.6135%
genWin : 1m 53s (15 30%) loss : 0.2031, accuracy : 40.1757%
genWin : 2m 28s (20 40%) loss : 0.2032, accuracy : 39.8173%
genWin : 3m 3s (25 50%) loss : 0.2031, accuracy : 39.5924%
genWin : 3m 39s (30 60%) loss : 0.2031, accuracy : 40.2600%
genWin : 4m 14s (35 70%) loss : 0.2031, accuracy : 40.2881%
genWin : 4m 49s (40 80%) loss : 0.2030, accuracy : 40.6184%
genWin : 5m 24s (45 90%) loss : 0.2031, accuracy : 39.9859%
Finished Training


[0.3985125322089482,
 0.3961349262122277,
 0.4017568517217147,
 0.3981728742094167,
 0.39592410400562195,
 0.40260014054813775,
 0.4028812368236121,
 0.4061841180604357,
 0.39985945186226285]

In [98]:
paramsDisOnGen['epochs']=5
paramsDisOnGen['print_every']=1
paramsDisOnGen['learning_rate_discriminator']=5e-5
paramsDisOnGen['dropout_discriminator']=0.1
paramsDisOnGen['genNet']=myGenerator

myDis.trainOnGen(**paramsDisOnGen)

Trying using Cuda ...
OK
Start training
disOnGen : 0m 13s (1 20%) loss : 0.1982, accuracy : 79.9368%
disOnGen : 0m 19s (2 40%) loss : 0.1941, accuracy : 98.1026%
disOnGen : 0m 25s (3 60%) loss : 0.1915, accuracy : 99.7540%
disOnGen : 0m 32s (4 80%) loss : 0.1888, accuracy : 99.9649%
Finished Training


[0.7993675333801827, 0.9810260014054814, 0.9975404075895994, 0.999648629655657]

In [99]:
paramsDisOnData['epochs']=5
paramsDisOnData['print_every']=1
paramsDisOnData['learning_rate_discriminator']=2e-5
paramsDisOnData['dropout_discriminator']=0.1

myDis.trainOnData(**paramsDisOnData)

Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 20%) loss : 0.2162, accuracy : 1.8271%
disOnDataWin : 0m 2s (2 40%) loss : 0.2145, accuracy : 4.4273%
disOnDataWin : 0m 3s (3 60%) loss : 0.2135, accuracy : 6.3598%
disOnDataWin : 0m 4s (4 80%) loss : 0.2121, accuracy : 8.3626%
Finished Training


[0.018271257905832748,
 0.04427266338721012,
 0.06359803232607168,
 0.08362614195361912]

In [122]:
# Train discriminator
disOnGenWin = 1
disOnDataWin = 1
metaEpochs = 50
paramsDisOnGen['epochs']=2
paramsDisOnData['epochs']=2

for i in range(metaEpochs):
    print("\niteration: %d\n" % i)
    if disOnDataWin>0.75 or disOnDataWin>disOnGenWin:
        paramsDisOnGen['epochs']=4
        paramsDisOnData['epochs']=4

    if disOnGenWin>0.75 or disOnGenWin>disOnDataWin:
        paramsDisOnGen['epochs']=2
        paramsDisOnData['epochs']=7
        
    disOnGenWin = max(myDis.trainOnGen(**paramsDisOnGen))
    disOnDataWin = max(myDis.trainOnData(**paramsDisOnData))






iteration: 0

Trying using Cuda ...
OK
Start training
disOnGen : 0m 13s (1 50%) loss : 0.2013, accuracy : 56.1665%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 14%) loss : 0.2023, accuracy : 50.8609%
disOnDataWin : 0m 2s (2 28%) loss : 0.2004, accuracy : 59.4870%
disOnDataWin : 0m 3s (3 42%) loss : 0.1998, accuracy : 62.1574%
disOnDataWin : 0m 4s (4 57%) loss : 0.1986, accuracy : 64.9684%
disOnDataWin : 0m 5s (5 71%) loss : 0.1971, accuracy : 70.9768%
disOnDataWin : 0m 6s (6 85%) loss : 0.1959, accuracy : 76.1068%
Finished Training

iteration: 1

Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 25%) loss : 0.2034, accuracy : 37.7723%
disOnGen : 0m 19s (2 50%) loss : 0.1992, accuracy : 74.0337%
disOnGen : 0m 25s (3 75%) loss : 0.1964, accuracy : 90.0562%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 25%) loss : 0.2057, accuracy : 37.6669%
disOnDataWin : 0m 2s (2 50%) loss : 0.2040, accuracy : 44.3078%


disOnDataWin : 0m 6s (5 71%) loss : 0.2010, accuracy : 55.4462%
disOnDataWin : 0m 7s (6 85%) loss : 0.1998, accuracy : 60.1546%
Finished Training

iteration: 15

Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 50%) loss : 0.1994, accuracy : 71.3457%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 14%) loss : 0.2037, accuracy : 45.7484%
disOnDataWin : 0m 2s (2 28%) loss : 0.2022, accuracy : 51.0541%
disOnDataWin : 0m 3s (3 42%) loss : 0.2005, accuracy : 57.5193%
disOnDataWin : 0m 5s (4 57%) loss : 0.1999, accuracy : 59.9086%
disOnDataWin : 0m 5s (5 71%) loss : 0.1985, accuracy : 65.6360%
disOnDataWin : 0m 6s (6 85%) loss : 0.1979, accuracy : 67.0063%
Finished Training

iteration: 16

Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 50%) loss : 0.2017, accuracy : 52.8988%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 14%) loss : 0.2011, accuracy : 55.1124%
disOnDataWin : 0m 3s (2 28%) loss : 0.

disOnDataWin : 0m 1s (1 25%) loss : 0.2043, accuracy : 42.1117%
disOnDataWin : 0m 3s (2 50%) loss : 0.2026, accuracy : 48.6297%
disOnDataWin : 0m 3s (3 75%) loss : 0.2015, accuracy : 52.0028%
Finished Training

iteration: 30

Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 50%) loss : 0.1972, accuracy : 84.9965%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 14%) loss : 0.2051, accuracy : 38.6156%
disOnDataWin : 0m 2s (2 28%) loss : 0.2036, accuracy : 44.8700%
disOnDataWin : 0m 4s (3 42%) loss : 0.2022, accuracy : 50.5271%
disOnDataWin : 0m 4s (4 57%) loss : 0.2013, accuracy : 53.5840%
disOnDataWin : 0m 5s (5 71%) loss : 0.2001, accuracy : 58.2221%
disOnDataWin : 0m 6s (6 85%) loss : 0.1988, accuracy : 61.9115%
Finished Training

iteration: 31

Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 50%) loss : 0.1995, accuracy : 70.1862%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 14%) loss : 0.

disOnGen : 0m 19s (2 50%) loss : 0.1982, accuracy : 79.3746%
disOnGen : 0m 26s (3 75%) loss : 0.1956, accuracy : 91.8131%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 25%) loss : 0.2052, accuracy : 37.8426%
disOnDataWin : 0m 2s (2 50%) loss : 0.2034, accuracy : 44.7997%
disOnDataWin : 0m 4s (3 75%) loss : 0.2020, accuracy : 51.1947%
Finished Training

iteration: 45

Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 50%) loss : 0.1957, accuracy : 90.2495%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 2s (1 14%) loss : 0.2062, accuracy : 32.9585%
disOnDataWin : 0m 3s (2 28%) loss : 0.2045, accuracy : 40.1968%
disOnDataWin : 0m 4s (3 42%) loss : 0.2036, accuracy : 45.8890%
disOnDataWin : 0m 5s (4 57%) loss : 0.2024, accuracy : 49.6135%
disOnDataWin : 0m 6s (5 71%) loss : 0.2010, accuracy : 54.4273%
disOnDataWin : 0m 6s (6 85%) loss : 0.2000, accuracy : 57.7301%
Finished Training

iteration: 46

Trying using Cuda

In [96]:
#Train Global
paramsGen['epochs']=2
paramsDisOnGen['epochs']=2
paramsDisOnData['epochs']=2

metaEpochs = 5

genWin = 0
disOnGenWin = 0
disOnDataWin = 0

for i in range(metaEpochs):
    if disOnGenWin < 0.75:
        disOnGenWin = max(myDis.trainOnGen(**paramsDisOnGen))
    if disOnDataWin < 0.75:
        disOnDataWin = max(myDis.trainOnData(**paramsDisOnData))
    else:
        genWin = max(myGenerator.trainOnDis(**paramsGen))

print("finished")



Trying using Cuda ...
OK
Start training
disOnGen : 0m 12s (1 50%) loss : 0.0005, accuracy : 100.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 50%) loss : 0.7791, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
genWin : 0m 18s (1 50%) loss : 0.7450, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 50%) loss : 0.7085, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
genWin : 0m 18s (1 50%) loss : 0.6787, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 50%) loss : 0.6441, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
genWin : 0m 18s (1 50%) loss : 0.6162, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
disOnDataWin : 0m 1s (1 50%) loss : 0.5820, accuracy : 0.0000%
Finished Training
Trying using Cuda ...
OK
Start training
genWin :

In [124]:
test_sequence = ["C:maj","F:maj","C:maj","C:maj",
                 "F:maj","F:maj","C:maj","C:maj",
                 "G:maj","F:maj","C:maj","G:maj",
                 "C:maj","C:maj","F:maj","G:maj"]
myGenerator.generateFromSequence(test_sequence, generation_lenght=64,
                               alphabet = 'a0',sampling=True, 
                               using_cuda=True, silent=True)

Trying using Cuda ...
OK
['C:maj', 'F:maj', 'C:maj', 'C:maj']
['F:maj', 'F:maj', 'C:maj', 'C:maj']
['G:maj', 'F:maj', 'C:maj', 'G:maj']
['C:maj', 'C:maj', 'F:maj', 'G:maj']
generated :
['N', 'G:maj', 'D#:maj', 'A#:maj']
['A:min', 'C:min', 'G:min', 'C#:min']
['C:maj', 'A:maj', 'F#:min', 'F#:maj']
['G#:maj', 'G#:maj', 'E:min', 'F:maj']
['A#:min', 'A#:maj', 'D#:maj', 'D#:maj']
['C:maj', 'N', 'A#:min', 'D:maj']
['N', 'B:maj', 'E:maj', 'A#:min']
['A:min', 'D#:maj', 'G:maj', 'N']
['C#:maj', 'G#:maj', 'G#:maj', 'A#:min']
['D#:maj', 'E:maj', 'N', 'E:min']
['F:maj', 'B:min', 'G#:min', 'C#:min']
['G#:min', 'F:maj', 'N', 'D#:min']


In [123]:
using_cuda = True
# Cuda blabla
if using_cuda:
    print("Trying using Cuda ...")
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        print("OK")
    else:
        print("Woops, Cuda cannot be found :'( ")
    device = torch.device("cuda:0" if use_cuda else "cpu")
else:
    device = torch.device("cpu")
    print("Using Cpu")

lenSeq = len(test_sequence)


# Getting chords dictionary
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
dictChord, listChord = chordUtil.getDictChord(eval(alphabet))


# Initialising objects
test_sequence_tensor = torch.zeros(1, len(test_sequence), len(dictChord)).to(device)
last_chords_output = torch.zeros(1, lenSeq, len(dictChord)).to(device)
test_sequence_tensor.requires_grad = False
last_chords_output.requires_grad = False
for t in range(len(test_sequence)):
    test_sequence_tensor[0, t, dictChord[test_sequence[t]]] = 1
    
#print(test_sequence_tensor.cpu())

myDis(test_sequence_tensor)

Trying using Cuda ...
OK


tensor([[0.5141]], device='cuda:0', grad_fn=<ThAddmmBackward>)